<a href="https://colab.research.google.com/github/duonghung86/Fatality-crashes/blob/master/VCA_2_1_MLP_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [95]:
import warnings
warnings.filterwarnings("ignore")

In [96]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [97]:
import psutil
# gives a single float value
print(psutil.cpu_percent())
# gives an object with many fields
psutil.virtual_memory()

18.2


svmem(total=13653585920, available=12362915840, percent=9.5, used=1665130496, free=9322291200, active=1128464384, inactive=2923970560, buffers=102735872, cached=2563428352, shared=954368, slab=179167232)

In [98]:
# Basic packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
import os
# Preprocessing
from sklearn.preprocessing import StandardScaler # Standardization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Machine learning algos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from imblearn.over_sampling import SMOTE, RandomOverSampler,BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler,NearMiss,EditedNearestNeighbours
# Metrics
from imblearn.metrics import geometric_mean_score
# Tensorflow
import tensorflow as tf
print(tf.__version__)
from tensorflow import feature_column  # for data wrangling
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.metrics import CohenKappa,F1Score
from tensorboard.plugins.hparams import api as hp

2.3.0


# Load data

In [99]:
url = 'https://github.com/duonghung86/Injury-severity-classification/blob/main/Prepared%20Texas%202019.zip?raw=true' 
data_path = tf.keras.utils.get_file(origin=url, fname=url.split('/')[-1].split('?')[0], extract=True)
data_path = data_path.replace('%20',' ').replace('.zip','.csv')

10592256/10587626 [==============================] - 0s 0us/step


In [100]:
# Load data
df = pd.read_csv(data_path)
print(df.shape)
df.head(3)

(949856, 19)


,Prsn_Injry_Sev,Prsn_Age,Prsn_Gndr,Wthr_Cond,Light_Cond,Surf_Cond,Veh_Body_Styl,Prsn_Rest,Prsn_Drg_Rslt,Harm_Evnt,Rural,Crash_Speed_Limit,Road_Algn,Veh_Mod_Year,Weekend,Crash_season,Part_of_day,Collsn_type,Collsn_name
0,0,26,MALE,CLEAR,DAYLIGHT,DRY,"PASSENGER CAR, 2-DOOR",SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,0,-1,"STRAIGHT, LEVEL",33,1,3,3,SAME DIRECTION,ONE STRAIGHT-ONE LEFT TURN
1,0,52,MALE,CLEAR,DAYLIGHT,DRY,"PASSENGER CAR, 2-DOOR",SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,0,-1,"STRAIGHT, LEVEL",19,1,3,3,SAME DIRECTION,ONE STRAIGHT-ONE LEFT TURN
2,0,27,FEMALE,CLEAR,DAYLIGHT,DRY,PICKUP,SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,1,-1,"CURVE, LEVEL",16,1,3,4,SAME DIRECTION,BOTH LEFT TURN


In [101]:
# Let's just use 80% of the total dataset
df, _ = train_test_split(df, test_size=0.8,stratify = df['Prsn_Injry_Sev'])
df.shape

(189971, 19)

In [102]:
y = df['Prsn_Injry_Sev']
print('All target values:')
print(y.value_counts())
X = df.drop(columns=['Prsn_Injry_Sev'])

All target values:
0    158512
1     20482
2      9048
3      1590
4       339
Name: Prsn_Injry_Sev, dtype: int64


In [103]:
# %% Data wrangling -------------
# Classify variable type
emb_vars, ind_vars, num_vars = [], [], []
for var in X.columns:
    if X[var].dtypes == 'O':
        if len(X[var].unique()) > 5:
            emb_vars.append(var)
        else:
            ind_vars.append(var)
    else:
        num_vars.append(var)
print('Numerical variables are ', num_vars)
print('Categorical variables that have at most 5 categories are ', ind_vars)
print('Categorical variables that have more than 5 categories are ', emb_vars)

# Create feature columns
feature_columns = []
# numeric cols
for header in num_vars:
    feature_columns.append(feature_column.numeric_column(header))
# bucketized cols
# age = feature_column.numeric_column('Prsn_Age')
# age_buckets = feature_column.bucketized_column(age, boundaries=[16, 22, 35, 55, 65])
# feature_columns.append(age_buckets)
# indicator_columns
for col_name in ind_vars:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
        col_name, X[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
# embedding columns
for col_name in emb_vars:
    emb_column = feature_column.categorical_column_with_vocabulary_list(
        col_name, X[col_name].unique())
    col_embedding = feature_column.embedding_column(emb_column, dimension=5)
    feature_columns.append(col_embedding)

# Convert all setup into new dataset
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
X = feature_layer(dict(X)).numpy()
print('New shape of the input data set:',X.shape)

Numerical variables are  ['Prsn_Age', 'Rural', 'Crash_Speed_Limit', 'Veh_Mod_Year', 'Weekend', 'Crash_season', 'Part_of_day']
Categorical variables that have at most 5 categories are  ['Prsn_Gndr', 'Prsn_Drg_Rslt', 'Collsn_type']
Categorical variables that have more than 5 categories are  ['Wthr_Cond', 'Light_Cond', 'Surf_Cond', 'Veh_Body_Styl', 'Prsn_Rest', 'Harm_Evnt', 'Road_Algn', 'Collsn_name']
New shape of the input data set: (189971, 59)


In [104]:
# %% Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=48)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=48)

print('Training features shape:', X_train.shape)
print('Validation features shape:', X_val.shape)
print('Test features shape:', X_test.shape)

# %% standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Training features shape: (121580, 59)
Validation features shape: (30396, 59)
Test features shape: (37995, 59)


In [105]:
# %% Function to compare the prediction and true labels
def get_accs(label, prediction, tr_time,  show=False):
    cm = confusion_matrix(label, prediction)
    length = cm.shape[0]
    num_cases = len(label)
    # global accuracy
    glb_acc = np.trace(cm) / len(label)
    ind_accs = cm / np.sum(cm, axis=1)[:, np.newaxis]
    accs = [ind_accs[i, i] for i in range(length)]
    index = ['Class {}'.format(i) for i in range(length)]
    # Global accuracy
    accs.append(glb_acc)
    #index.append
    # G-mean
    accs.append(geometric_mean_score(label, prediction, correction=0.001))
    #index.append('G-mean')
    # Average perf
    accs.append((glb_acc + accs[-1]) / 2)
    #index.append('Avg_Pfm')
    # Training time
    accs.append(np.round(tr_time,3))
    index = index + ['Overall Accuracy','G-mean','Avg_Pfm','Training Time']
    # Plot confusion matrix
    plot_dict = {'Confusion matrix': (cm,'g'),
                 'Normalized confusion matrix': (ind_accs,'.2f')}
    if show:
        plt.figure(figsize=(14, 6))
        i = 1
        for key, value in plot_dict.items():
            plt.subplot(1, 2, i)
            sns.heatmap(value[0], xticklabels=np.arange(length), yticklabels=np.arange(length),
                        annot=True, fmt=value[1], cmap="YlGnBu")
            plt.xlabel('Prediction')
            plt.ylabel('Label')
            plt.title(key)
            i+= 1
        plt.show()
    out = np.array(accs).reshape(1, len(accs))
    return pd.DataFrame(out, columns=index)

# Class weights

In [106]:
wgt='balanced'
clfs = [LogisticRegression(solver = 'lbfgs',class_weight=wgt),
        DecisionTreeClassifier(class_weight=wgt),
        RandomForestClassifier(max_depth=4,class_weight=wgt)]
clf_names = ['LR','DT','RF']
rsts = pd.DataFrame()
for model, name in zip(clfs,clf_names):
    start = time.time()
    print(name)
    model.fit(X_train, y_train.values)
    # use the model to make predictions with the test data
    y_pred = model.predict(X_test)
    end= time.time()
    # get the evaluation metrics
    result = get_accs(y_test.values, y_pred, end-start)
    result.index = [name + '-Weights']
    rsts = rsts.append(result)
rsts

LR
DT
RF


,Class 0,Class 1,Class 2,Class 3,Class 4,Overall Accuracy,G-mean,Avg_Pfm,Training Time
LR-Weights,0.522474,0.422363,0.149171,0.349057,0.676471,0.492723,0.378544,0.435633,12.408
DT-Weights,0.827997,0.135986,0.084530,0.066038,0.161765,0.710409,0.159017,0.434713,2.753
RF-Weights,0.578305,0.388916,0.066851,0.298742,0.705882,0.531412,0.316395,0.423904,8.283


## MLP

In [107]:
# Add weights
weights = len(y) / (5 * np.bincount(y))
cls_wgt = dict(zip(np.arange(5), weights))
cls_wgt

{0: 0.23969289391339457,
 1: 1.8550043941021384,
 2: 4.199182139699381,
 3: 23.89572327044025,
 4: 112.07728613569321}

In [108]:
es = EarlyStopping(monitor='val_cohen_kappa',
                   verbose=1, patience=10, mode='max',
                   restore_best_weights=True)

In [109]:
def create_mlp():
    MLP = Sequential([Dense(10,activation='relu',
                           input_dim=X_train.shape[1]),
                      Dropout(0.5),
                      Dense(5, activation='softmax')])
    MLP.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[CohenKappa(num_classes=5,sparse_labels=True)])
    return MLP

In [111]:
model = create_mlp()
start = time.time()
monitor = model.fit(X_train, y_train.values,
                    callbacks=[es],
                    class_weight = cls_wgt,
                    validation_data=(X_val, y_val.values),
                    verbose=1, epochs=50)
end = time.time()
# use the model to make predictions with the test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
# get the evaluation metrics
result = get_accs(y_test.values, y_pred, end-start)
result.index = ['MLP-Weights']
rsts = rsts.append(result)
rsts

Epoch 1/50
3800/3800 [==============================] - 6s 2ms/step - loss: 1.5631 - cohen_kappa: 0.0048 - val_loss: 1.5276 - val_cohen_kappa: 0.0411
Epoch 2/50
3800/3800 [==============================] - 6s 2ms/step - loss: 1.5221 - cohen_kappa: 0.0416 - val_loss: 1.4543 - val_cohen_kappa: 0.0643
Epoch 3/50
3800/3800 [==============================] - 6s 2ms/step - loss: 1.5012 - cohen_kappa: 0.0519 - val_loss: 1.4593 - val_cohen_kappa: 0.0715
Epoch 4/50
3800/3800 [==============================] - 6s 2ms/step - loss: 1.4951 - cohen_kappa: 0.0569 - val_loss: 1.4481 - val_cohen_kappa: 0.0800
Epoch 5/50
3800/3800 [==============================] - 6s 2ms/step - loss: 1.4858 - cohen_kappa: 0.0603 - val_loss: 1.4337 - val_cohen_kappa: 0.0845
Epoch 6/50
3800/3800 [==============================] - 6s 2ms/step - loss: 1.4844 - cohen_kappa: 0.0640 - val_loss: 1.4574 - val_cohen_kappa: 0.0798
Epoch 7/50
3800/3800 [==============================] - 6s 2ms/step - loss: 1.4788 - cohen_kappa: 0.

,Class 0,Class 1,Class 2,Class 3,Class 4,Overall Accuracy,G-mean,Avg_Pfm,Training Time
LR-Weights,0.522474,0.422363,0.149171,0.349057,0.676471,0.492723,0.378544,0.435633,12.408
DT-Weights,0.827997,0.135986,0.084530,0.066038,0.161765,0.710409,0.159017,0.434713,2.753
RF-Weights,0.578305,0.388916,0.066851,0.298742,0.705882,0.531412,0.316395,0.423904,8.283
MLP-Weights,0.528373,0.417480,0.195028,0.229560,0.691176,0.498329,0.368834,0.433582,129.129


# Normal Resampling

In [ ]:
Resamples = {'ROS':             RandomOverSampler(), 
             'SMOTE':           SMOTE(random_state=42),
             'BorderlineSMOTE': BorderlineSMOTE(),
             'RUS': RandomUnderSampler(), 
             'NearMiss': NearMiss()
             }
for key,value in Resamples.items():
    start = time.time()
    X_res, y_res = value.fit_resample(X_train, y_train)
    end = time.time()
    res_time = end - start
    print(key)
    print('Resampled dataset shape %s' % Counter(y_res))
    print('resampling time is {0:.2f} seconds'.format(res_time))
    
    #Logistic model
    LR = LogisticRegression(solver = 'lbfgs')
    start = time.time()
    LR.fit(X_res, y_res)
    end= time.time()
    # get the evaluation metrics
    # use the model to make predictions with the test data
    y_pred = LR.predict(X_test)
    result = get_accs(y_test.values,y_pred, tr_time= end-start)
    result['Resample time'] = res_time
    result.index = ['LR-' + key]
    rsts = rsts.append(result)
    
    # MLP model
    model = create_mlp()
    start = time.time()
    monitor = model.fit(X_res, y_res,
                        callbacks=[es],
                        validation_data=(X_val, y_val.values),
                        verbose=1, epochs=50)
    end = time.time()
    # use the model to make predictions with the test data
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    # get the evaluation metrics
    result = get_accs(y_test.values, y_pred, end-start)
    result.index = ['MLP-'+ key]
    rsts = rsts.append(result)
rsts

ROS
Resampled dataset shape Counter({1: 101446, 0: 101446, 2: 101446, 3: 101446, 4: 101446})
resampling time is 0.14 seconds
Epoch 1/50
15851/15851 [==============================] - 21s 1ms/step - loss: 1.4708 - cohen_kappa: 0.2496 - val_loss: 1.4256 - val_cohen_kappa: 0.0864
Epoch 2/50
15851/15851 [==============================] - 21s 1ms/step - loss: 1.4484 - cohen_kappa: 0.2818 - val_loss: 1.4149 - val_cohen_kappa: 0.0882
Epoch 3/50
15851/15851 [==============================] - 21s 1ms/step - loss: 1.4439 - cohen_kappa: 0.2890 - val_loss: 1.4197 - val_cohen_kappa: 0.0869
Epoch 4/50
  846/15851 [>.............................] - ETA: 18s - loss: 1.4447 - cohen_kappa: 0.2893

In [ ]:
#rsts = rsts.iloc[:9,:]
#rsts

# Hybrid Resampling


In [ ]:
y_dict = Counter(y_train)
y_dict

## Oversampling and then undersampling

In [ ]:
ss = {}
for i in range(2,5):
    ss[i] = y_dict[1]
oses = ['ROS','SMOTE','BorderlineSMOTE']
uses = ['RUS','NearMiss']

In [ ]:
for os_name in oses:
    for us_name in uses:
        print(os_name,'-',us_name)
        if os_name == 'ROS': 
            res = RandomOverSampler(sampling_strategy=ss)
        elif os_name == 'BorderlineSMOTE': 
            res = BorderlineSMOTE(sampling_strategy=ss)
        else: res = SMOTE(sampling_strategy=ss)
        start = time.time()
        X_res, y_res = res.fit_resample(X_train, y_train)

        if us_name == 'RUS': 
            res = RandomUnderSampler(sampling_strategy='majority')
        else: res = NearMiss(sampling_strategy='majority')  
        X_res, y_res = res.fit_resample(X_res, y_res)
        end = time.time()
        res_time = end-start

        print('Resampled dataset shape %s' % Counter(y_res))
        print('Resamling time %.2f sec' % (res_time))
        LR = LogisticRegression(solver = 'lbfgs')
        start = time.time()
        LR.fit(X_res, y_res)
        end= time.time()
        # get the evaluation metrics
        # use the model to make predictions with the test data
        y_pred = LR.predict(X_test)
        result = get_accs(y_test.values,y_pred, tr_time= end-start)
        result['Resample time'] = res_time
        result.index = ['LR-' + os_name + '-' + us_name]
        rsts = rsts.append(result)

        # MLP model
        model = create_mlp()
        start = time.time()
        monitor = model.fit(X_res, y_res,
                            callbacks=[es],
                            validation_data=(X_val, y_val.values),
                            verbose=1, epochs=50)
        end = time.time()
        # use the model to make predictions with the test data
        y_pred = model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        # get the evaluation metrics
        result = get_accs(y_test.values, y_pred, end-start)
        result.index = ['MLP-'+ key]
        rsts = rsts.append(result)
rsts

## Under sampling and then over sampling

In [ ]:
for us_name in uses:
    for os_name in oses:
        print(us_name,'-',os_name)
        if us_name == 'RUS': 
            res = RandomUnderSampler(sampling_strategy={0: y_dict[1]})
        else: res = NearMiss(sampling_strategy={0: y_dict[1]})
        start = time.time()
        X_res, y_res = res.fit_resample(X_train, y_train)

        if os_name == 'ROS': 
            res = RandomOverSampler(sampling_strategy='not majority')
        elif os_name == 'BorderlineSMOTE': 
            res = BorderlineSMOTE(sampling_strategy='not majority')
        else: res = SMOTE(sampling_strategy='not majority')
        X_res, y_res = res.fit_resample(X_res, y_res)
        end = time.time()
        res_time = end-start
        print('Resampled dataset shape %s' % Counter(y_res))
        print('Resamling time %.2f sec' % (res_time))
        LR = LogisticRegression(solver = 'lbfgs',
                                #class_weight= 'balanced'
                                )
        start = time.time()
        LR.fit(X_res, y_res)
        end= time.time()
        # get the evaluation metrics
        # use the model to make predictions with the test data
        y_pred = LR.predict(X_test)
        result = get_accs(y_test.values,y_pred, tr_time= end-start)
        result['Resample time'] = res_time
        result.index = ['LR-' + us_name + '-' + os_name]
        rsts = rsts.append(result)

        # MLP model
        model = create_mlp()
        start = time.time()
        monitor = model.fit(X_res, y_res,
                            callbacks=[es],
                            validation_data=(X_val, y_val.values),
                            verbose=1, epochs=50)
        end = time.time()
        # use the model to make predictions with the test data
        y_pred = model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        # get the evaluation metrics
        result = get_accs(y_test.values, y_pred, end-start, True)
        result.index = ['MLP-'+ key]
        rsts = rsts.append(result)
rsts

### Refining result
NearMiss not only increased the computing time but also reduced the accuracy significantly
Apply 5-fold cross validation


In [ ]:
for

# ALL mini functions



In [ ]:
def show_evolution(moni):
    hist = pd.DataFrame(monitor.history)
    no_metrics = np.int(hist.shape[1]/2)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 2), dpi=150)
    for i in range(2):
      hist.iloc[:,[i,no_metrics+i]].plot(ax=axes[i])
    plt.show()
